### Importações

In [8]:
import xml.etree.ElementTree as ET
import pandas as pd
import re

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

#### Leitura dos arquivos

In [ ]:
# Caminho do arquivo XML exportado
xml_file = r"C:\Users\breno\OneDrive\Documentos\Python_Projects\pcp_pmp\data\New_Project.xml"

# Parse do XML
tree = ET.parse(xml_file)
root = tree.getroot()

# Namespace do XML
ns = {"msproj": "http://schemas.microsoft.com/project"}

# Inicializar um conjunto para armazenar colunas únicas
colunas_unicas = set()

# Função recursiva para percorrer todos os nós do XML
def extrair_colunas(elemento, prefixo=""):
    for subelemento in elemento:
        nome_coluna = prefixo + subelemento.tag.replace("{http://schemas.microsoft.com/project}", "")
        colunas_unicas.add(nome_coluna)  # Adiciona ao conjunto (sem duplicação)
        extrair_colunas(subelemento, prefixo=nome_coluna + "/")  # Chamada recursiva

# Percorrer todas as tarefas e identificar colunas
tasks = []
for task in root.findall(".//msproj:Task", ns):
    task_data = {}
    extrair_colunas(task)
    for subelemento in task:
        nome_coluna = subelemento.tag.replace("{http://schemas.microsoft.com/project}", "")
        task_data[nome_coluna] = subelemento.text
    tasks.append(task_data)

# Identificar ExtendedAttributes
for ext_attr in root.findall(".//msproj:ExtendedAttribute", ns):
    field_id = ext_attr.find("msproj:FieldID", ns)
    if field_id is not None:
        colunas_unicas.add(f"Extended_{field_id.text}")  # Nome dinâmico para ExtendedAttributes

# Ordenar e exibir todas as colunas encontradas
colunas_ordenadas = sorted(colunas_unicas)
#print(f"Total de colunas encontradas: {len(colunas_ordenadas)}\n")
#print("\n".join(colunas_ordenadas))

# Converter para DataFrame
df = pd.DataFrame(tasks)


# Exibir as primeiras 10 linhas do DataFrame
display(df.head(10))

# Exibir a quantidade de colunas
print(f"Quantidade de colunas: {len(df.columns)}")

### Renamed Columns

In [ ]:
# Dicionário de tradução (inglês -> português)
traducao_colunas = {
    "ACWP": "Custo Real do Trabalho Executado",
    "Active": "Ativo",
    "ActualCost": "Custo Real",
    "ActualDuration": "Duração Real",
    "ActualFinish": "Término Real",
    "ActualOvertimeCost": "Custo Real de Hora Extra",
    "ActualOvertimeWork": "Trabalho Real de Hora Extra",
    "ActualStart": "Início Real",
    "ActualWork": "Trabalho Real",
    "BCWP": "Custo Orçado do Trabalho Executado",
    "BCWS": "Custo Orçado do Trabalho Planejado",
    "CV": "Variação de Custo",
    "CalendarUID": "UID do Calendário",
    "CommitmentType": "Tipo de Compromisso",
    "ConstraintDate": "Data de Restrição",
    "ConstraintType": "Tipo de Restrição",
    "Cost": "Custo",
    "CreateDate": "Data de Criação",
    "Critical": "Crítico",
    "DisplayAsSummary": "Exibir como Resumo",
    "Duration": "Duração",
    "DurationFormat": "Formato de Duração",
    "EarlyFinish": "Término Antecipado",
    "EarlyStart": "Início Antecipado",
    "EarnedValueMethod": "Método de Valor Agregado",
    "EffortDriven": "Dirigido por Esforço",
    "Estimated": "Estimado",
    "ExtendedAttribute": "Atributo Estendido",
    "ExtendedAttribute/FieldID": "ID do Campo do Atributo Estendido",
    "ExtendedAttribute/Value": "Valor do Atributo Estendido",
    "ExternalTask": "Tarefa Externa",
    "Finish": "Término",
    "FinishSlack": "Folga de Término",
    "FinishVariance": "Variação de Término",
    "FixedCost": "Custo Fixo",
    "FixedCostAccrual": "Acúmulo de Custo Fixo",
    "FreeSlack": "Folga Livre",
    "FreeformDurationFormat": "Formato de Duração Livre",
    "GUID": "GUID",
    "HideBar": "Ocultar Barra",
    "ID": "ID",
    "IgnoreResourceCalendar": "Ignorar Calendário de Recursos",
    "IsNull": "É Nulo",
    "IsPublished": "É Publicado",
    "IsSubproject": "É Subprojeto",
    "IsSubprojectReadOnly": "Subprojeto Somente Leitura",
    "LateFinish": "Término Tardio",
    "LateStart": "Início Tardio",
    "LevelAssignments": "Nivelar Atribuições",
    "LevelingCanSplit": "Nivelamento Pode Dividir",
    "LevelingDelay": "Atraso de Nivelamento",
    "LevelingDelayFormat": "Formato de Atraso de Nivelamento",
    "Manual": "Manual",
    "ManualDuration": "Duração Manual",
    "ManualFinish": "Término Manual",
    "ManualStart": "Início Manual",
    "Milestone": "Marco",
    "Name": "Nome",
    "OutlineLevel": "Nível de Estrutura",
    "OutlineNumber": "Número de Estrutura",
    "OverAllocated": "Sobrealocado",
    "OvertimeCost": "Custo de Hora Extra",
    "OvertimeWork": "Trabalho de Hora Extra",
    "PercentComplete": "Percentual Concluído",
    "PercentWorkComplete": "Percentual de Trabalho Concluído",
    "PhysicalPercentComplete": "Percentual Físico Concluído",
    "PredecessorLink": "Link de Predecessor",
    "PredecessorLink/CrossProject": "Link de Predecessor/Projeto Cruzado",
    "PredecessorLink/LagFormat": "Link de Predecessor/Formato de Atraso",
    "PredecessorLink/LinkLag": "Link de Predecessor/Atraso de Link",
    "PredecessorLink/PredecessorUID": "Link de Predecessor/UID do Predecessor",
    "PredecessorLink/Type": "Link de Predecessor/Tipo",
    "Priority": "Prioridade",
    "Recurring": "Recorrente",
    "RegularWork": "Trabalho Regular",
    "RemainingCost": "Custo Restante",
    "RemainingDuration": "Duração Restante",
    "RemainingOvertimeCost": "Custo Restante de Hora Extra",
    "RemainingOvertimeWork": "Trabalho Restante de Hora Extra",
    "RemainingWork": "Trabalho Restante",
    "Resume": "Retomar",
    "ResumeValid": "Retomada Válida",
    "Rollup": "Agregação",
    "Start": "Início",
    "StartSlack": "Folga de Início",
    "StartVariance": "Variação de Início",
    "Stop": "Parar",
    "Summary": "Resumo",
    "TimephasedData": "Dados em Fase Temporal",
    "TimephasedData/Finish": "Dados em Fase Temporal/Término",
    "TimephasedData/Start": "Dados em Fase Temporal/Início",
    "TimephasedData/Type": "Dados em Fase Temporal/Tipo",
    "TimephasedData/UID": "Dados em Fase Temporal/UID",
    "TimephasedData/Unit": "Dados em Fase Temporal/Unidade",
    "TimephasedData/Value": "Dados em Fase Temporal/Valor",
    "TotalSlack": "Folga Total",
    "Type": "Tipo",
    "UID": "UID",
    "WBS": "EAP",
    "Work": "Trabalho",
    "WorkVariance": "Variação de Trabalho"
}

# Criar uma cópia do DataFrame
df_translater = df.copy()

# Renomear as colunas usando o dicionário de tradução
df_translater.rename(columns=traducao_colunas, inplace=True)

# Exibir as primeiras 10 linhas do DataFrame traduzido
display(df_translater.head(10))

# Exibir as colunas traduzidas
print("Colunas traduzidas:")
print(df_translater.columns.tolist())

In [ ]:
df_dias = df_translater.copy()

# Função para converter PTxHyMzS para dias úteis
def convert_pt_to_days(duration):
    if pd.isna(duration) or duration is None or duration == 'PT0H0M0S':
        return 0
    
    try:
        duration_str = str(duration)  # Ensure duration is a string
        match = re.match(r'PT(?:(\d+)H)?(?:(\d+)M)?(?:(\d+)S)?', duration_str)
        if not match:
            return 0
        
        hours = int(match.group(1)) if match.group(1) else 0
        minutes = int(match.group(2)) if match.group(2) else 0
        seconds = int(match.group(3)) if match.group(3) else 0
        
        total_hours = hours + (minutes / 60) + (seconds / 3600)
        days = total_hours / 8
        return days
    except (ValueError, AttributeError, TypeError):
        return 0  # Handle various potential errors


# Lista das colunas que precisam ser convertidas
duration_columns = [
    'Duração', 'Duração Manual', 'Trabalho', 'Trabalho de Hora Extra', 
    'Duração Real', 'Trabalho Real', 'Trabalho Real de Hora Extra', 
    'Trabalho Regular', 'Duração Restante', 'Trabalho Restante', 
    'Trabalho Restante de Hora Extra'
]

# Aplicar a função de conversão a cada coluna de duração
for col in duration_columns:
    if col in df_dias.columns:  # Check if column exists
        df_dias[col] = df_dias[col].apply(convert_pt_to_days)
    else:
        print(f"Column '{col}' not found in DataFrame.")


# Exibir o DataFrame com as novas colunas
display(df_dias.head(5))